# Задание **Lite**

1. Сделайте свою собственную архитектуру и запустите, оцените точность на проверочной выборке.

2. Возьмите лучшую архитектуру и проведите несколько экспериментов, меняя гиперпараметры выбранной архитектуры.

# Импорт бибилтотек

In [ ]:
!pip install -q pymorphy2

In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Embedding, Input, concatenate, Activation, MaxPooling1D, Conv1D
from tensorflow.keras.layers import BatchNormalization, Dropout, Conv1DTranspose, Lambda, Flatten, LeakyReLU, Reshape
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam, Adadelta
from tensorflow.keras.utils import plot_model

from gensim.models import word2vec # Импортируем gensim

import os
import re
from time import time
import numpy as np
import pandas as pd
from time import time
import nltk 
from nltk.stem import WordNetLemmatizer  
import pymorphy2 

nltk.download('wordnet') # Скачиваем сетку слов для лемматизации

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Функции для работы с текстами

In [ ]:
# Функция для удаления пунктуационных знаков препинания и еще дополнительных ненужных нам знаков из файла
def readText(fileName):
  f = open(fileName, 'r')
  text = f.read()
  delSymbols = ['\n', "\t", "\ufeff", ".", "_", "-", ",", "!", "?", "–", "(", ")", "«", "»", "№", ";",'•','%']

  for dS in delSymbols:
    text = text.replace(dS, " ")

  text = re.sub('[.]', ' ', text)
  text = re.sub(':', ' ', text)
  text = re.sub('<', ' <', text)
  text = re.sub('>', '> ', text)

  return ' '.join(text.split()).lower()

In [ ]:
# Функция которая конвертирует исходный текст в лист слов с начальной формой
def text2Words(text):
  morph = pymorphy2.MorphAnalyzer()
  words = text.split(" ")
  return [morph.parse(word)[0].normal_form for word in words]  

# Чтение файла

In [ ]:
directory = '/content/drive/MyDrive/УИИ/Cегментация текста/База договоров/'

In [ ]:
# Всего договоров
print(len(os.listdir(directory)), 'договора для обучения')

432 договора для обучения


In [ ]:
curTime = time()
agreements = [] # список договоров
for filename in os.listdir(directory):
  txt = readText(directory + filename)
  if txt != "":
    agreements.append(txt)
print(f'Загрузка файла заняла: {round(time() - curTime, 2)} с.')

Загрузка файла заняла: 1.14 с.


In [ ]:
# Конвертирует исходный текст в лист слов с начальной формой
docs_full = []
currTime = time()
for i in range(len(agreements)):
   docs_full.append(text2Words(agreements[i]))

print(f'Времени потребовалось: {round(time() - curTime, 2)} с.')

Времени потребовалось: 162.54 с.


In [ ]:
# Выводим число записей в наборе данных 
print(len(docs_full))

422


## Разделение на выборки

In [ ]:
# Выбираем итоговое количество данных для обучающей/проверочной и тестовой выборках
docsToTrain = docs_full[0:100]
docsToTest = docs_full[-10:]

In [ ]:
print('Число текстов в для тестовой проверки в конце ноутбука:', len(docsToTest)) 
print('Число договоров для обучающей и проверочной выборках:',len(docsToTrain)) 
print('Число слов в первом договоре:', len(docsToTrain[0]))

Число текстов в для тестовой проверки в конце ноутбука: 10
Число договоров для обучающей и проверочной выборках: 100
Число слов в первом договоре: 1149


# Превращение текстов в последовтельность индексов: создание xTrain

In [ ]:
tokenizer = Tokenizer(lower=True, filters='', char_level=False)
tokenizer.fit_on_texts(docs_full)
clean_voc = {} # Создаем пустой словарь

for i in tokenizer.word_index.items():
  clean_voc[i[0]] = i[1]

In [ ]:
# Превращает текст в последовательность индексов согласно словарю частотности
tok_agreem = tokenizer.texts_to_sequences(docsToTrain)

print("Исходный текст:              ", docsToTrain[50][:20])
print("Последовательность индексов: ", tok_agreem[50][:20])

Исходный текст:               ['договор', 'аренда', 'земельный', 'участок', 'находиться', 'в', 'государственный', 'или', 'муниципальный', 'собственность', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать', 'в', 'дальнейший']
Последовательность индексов:  [2, 116, 211, 169, 253, 1, 204, 16, 1287, 130, 76, 76, 1, 35, 38, 10, 65, 62, 1, 46]


# Создание yTrain

In [ ]:
#  Собираем лист индексов и их мультилейбл класссификации
def getXYSamples(tok_agreem, tags_index):
  tags01 = []
  indices = []

  for agreement in tok_agreem:
    tag_place = [0, 0, 0, 0, 0, 0]
    for ex in agreement:
      if ex in tags_index:
        place = np.argwhere(tags_index==ex)
        if len(place) !=0:
          if place[0][0] < 6:
            tag_place[place[0][0]] = 1
          else:
            tag_place[place[0][0] - 6] = 0
      else:
        tags01.append(tag_place.copy())
        indices.append(ex)
  return indices, tags01

In [ ]:
# Функция получение списка слов из списка индексов
def reverseIndex(clean_voc, x):
  reverse_word_map = dict(map(reversed, clean_voc.items()))
  docs = [reverse_word_map.get(letter) for letter in x]
  return docs

In [ ]:
#<s1> - Условия договора
#<s2> - Запреты
#<s3> - Стоимость
#<s4> - Условия (относительно дат)
#<s5> - Штрафы
#<s6> - Адреса и местоположения

tags_index = ['<s' + str(i) + '>' for i in range(1, 7)] # Лист открывающих тегов
closetags = ['</s' + str(i) + '>' for i in range(1, 7)] # Лист закрывающих тегов
tags_index.extend(closetags) # конкатенируем все теги

tags_index = np.array([clean_voc[i] for i in tags_index]) # Получаем инексы всех тегов из словря частотности
print('Индексы всех тегов:', tags_index)

Индексы всех тегов: [ 12 341  22  27 117 278  13 352  23  28 118 272]


In [ ]:
# Получаем теги и создаём листы с ними
curTime = time()
xData, yData = getXYSamples(tok_agreem,tags_index)
decoded_text = reverseIndex(clean_voc, xData) # Превращаем список индексов обратно в список слов
print(f'Время: {round(time() - curTime, 2)} с.')

Время: 0.45 с.


# Разделение выборки на окна

In [ ]:
# Создание выборки из индексов
def getSetFromIndices(wordIndices, xLen, step):
  xBatch = []
  wordsLen = len(wordIndices)
  index = 0
  while (index + xLen <= wordsLen):
    xBatch.append(wordIndices[index:index+xLen])
    index += step
  return xBatch

In [ ]:
xLen = 256
step = 30
embeddingSize = 300 # Пространство Embedding

In [ ]:
#  Генерируем выборки с параметрами наших окон
xTrain = getSetFromIndices(decoded_text, xLen, step)
yTrain = getSetFromIndices(yData, xLen, step)

### Текст в последовательность индексов

In [ ]:
tok_agreemTest = tokenizer.texts_to_sequences(docsToTest)

print("Посмотрим на фрагмент тестового текста:")
print()
print("Исходный текст:                                   ", docsToTest[5][:20])
print("Тот же текст, но как последовательность индексов: ", tok_agreemTest[5][:20])

Посмотрим на фрагмент тестового текста:

Исходный текст:                                    ['трудовой', 'контракт', 'с', 'специалист', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать', 'в', 'дальнейший', 'работодатель', 'с', 'один', 'сторона', 'и', 'гражданин']
Тот же текст, но как последовательность индексов:  [79, 187, 8, 897, 76, 76, 1, 35, 38, 10, 65, 62, 1, 46, 129, 8, 69, 7, 3, 265]


In [ ]:
# Распознаём теги и создаём лист, что их хранят
xDataTest, yDataTest = getXYSamples(tok_agreemTest, tags_index)

# Превращаем из списка индекса обратно в список слов
decoded_text = reverseIndex(clean_voc, xDataTest)

print('Длина xDataTest:', len(xDataTest))
print('Длина yDataTest:', len(yDataTest))
print()
print('Сдекодированные текст:', decoded_text[200:230])
print('Часть xDataTest:     ', xDataTest[200:230])
print('Часть yDataTest:     ', yDataTest[200:230])

Длина xDataTest: 10723
Длина yDataTest: 10723

Сдекодированные текст: ['2', 'основной', 'условие', 'договор', '2', '1', 'гр', 'назначаться', 'на', 'должность', 'директор', 'на', 'основание', 'решение', 'совет', 'директор', 'протокол', 'от', 'год', 'работодатель', 'поручать', 'директор', 'решение', 'весь', 'вопрос', 'текущий', 'деятельность', 'общество', 'за', 'исключение']
Часть xDataTest:      [4, 592, 40, 2, 4, 5, 523, 1530, 10, 495, 217, 10, 65, 280, 537, 217, 872, 29, 73, 129, 1005, 217, 280, 120, 248, 403, 148, 146, 17, 415]
Часть yDataTest:      [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [

In [ ]:
# Генерируем выборки
xTest = getSetFromIndices(decoded_text, xLen, step)
yTest = getSetFromIndices(yDataTest, xLen, step)

print('Длина xTest:', len(xTest))
print('Длина yTest:', len(yTest))
print()

print('Длина примера xTest:',len(xTest[10]))
print('Длина примера yTrain:',len(yTest[10]), '\n')

print('Пример xTest', xTest[10])
print('Пример yTest', yTest[10], '\n')

print('Первый пример xTest:', xTest[0][step-10:step+10])
print('Второй пример xTest:', xTest[1][:15])

Длина xTest: 349
Длина yTest: 349

Длина примера xTest: 256
Длина примера yTrain: 256 

Пример xTest ['договор', 'заключаться', 'на', 'неопределённый', 'срок', '3', '2', 'на', 'основание', 'решение', 'совет', 'директор', 'директор', 'издавать', 'приказ', 'о', 'свой', 'вступление', 'в', 'должность', '4', 'компетенция', 'директор', '4', '1', 'директор', 'решать', 'всё', 'вопрос', 'текущий', 'деятельность', 'банк', 'за', 'исключение', 'вопрос', 'отнести', 'к', 'исключительный', 'компетенция', 'общий', 'собрание', 'акционер', 'к', 'компетенция', 'совет', 'директор', 'или', 'правление', '4', '2', 'директор', 'без', 'доверенность', 'действовать', 'от', 'имя', 'банк', 'представлять', 'он', 'интерес', 'совершать', 'сделка', 'от', 'имя', 'банк', 'утверждать', 'штат', 'издавать', 'приказ', 'и', 'давать', 'указание', 'обязательный', 'для', 'исполнение', 'весь', 'работник', 'банк', '4', '3', 'право', 'и', 'обязанность', 'директор', 'по', 'осуществление', 'руководство', 'текущий', 'деятельность', '

# Создание xTrain и yTrain используя Word2Vec

In [ ]:
# Функция для дополнением нулями
def pad_zeros(phrase, xLen=256):
  while len(phrase) < xLen:
    phrase.append[0] * embeddingSize
  if len(phrase) > xLen:
    phrase = phrase[:xLen]
  return phrase

In [ ]:
# Создаём выборку
def getSets(model, senI, tagI):
  xVector = []
  tmp = []
  for text in senI:
    tmp=[]
    for word in text:
      try:
        tmp.append(model.wv[word])
      except:
        pass

    xVector.append(pad_zeros(tmp, xLen))
  temp = np.array(xVector)
  return np.array(xVector, dtype = np.float32), np.array(tagI)

In [ ]:
# Подаём лист листов слов в word2vec для обучающей выборке

# size = embeddingSize  - Размер ембеддинга
# window = 10           - Минимальное расстояния между эмбеддинг словами
# min_count = 1         - Игнорируем все слова с частотой меньше чем 1
# workers = 10          - Число потоков на обучение эмбеддинга
# iter = 20             - Число эпох на обучение эмбеддинга
# max_vocab_size = 1e5  - Число слов в "словарном запасе" word2vec

In [ ]:
curTime = time()

modelGENSIM = word2vec.Word2Vec(xTrain + xTest, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10, max_vocab_size = 1e5)
print(f'Обучение GENSIM заняло: {round(time() - curTime, 2)} с.')

Обучение GENSIM заняло: 13.41 с.


In [ ]:
GENSIMtrainX, GENSIMtrainY = getSets(modelGENSIM, xTrain, yTrain)
GENSIMtestX, GENSIMtestY = getSets(modelGENSIM, xTest, yTest)

In [ ]:
print('Размерности xTrain:', GENSIMtrainX.shape)
print('Размерности yTrain:', GENSIMtrainY.shape)

Размерности xTrain: (2645, 256, 300)
Размерности yTrain: (2645, 256, 6)


Сохранение массивов

In [ ]:
np.save('/content/drive/MyDrive/УИИ/Cегментация текста/GENSIMtrainX.npy', GENSIMtrainX)
np.save('/content/drive/MyDrive/УИИ/Cегментация текста/GENSIMtrainY.npy', GENSIMtrainY)

np.save('/content/drive/MyDrive/УИИ/Cегментация текста/GENSIMtestX.npy', GENSIMtestX)
np.save('/content/drive/MyDrive/УИИ/Cегментация текста/GENSIMtestY.npy', GENSIMtestY)

Загрузка

In [ ]:
# GENSIMtrainX = np.load('/content/drive/MyDrive/УИИ/Cегментация текста/GENSIMtrainX.npy')
# GENSIMtrainY = np.load('/content/drive/MyDrive/УИИ/Cегментация текста/GENSIMtrainY.npy')

# GENSIMtestX = np.load('/content/drive/MyDrive/УИИ/Cегментация текста/GENSIMtestX.npy')
# GENSIMtestY = np.load('/content/drive/MyDrive/УИИ/Cегментация текста/GENSIMtestY.npy')


# Функции создания моделей

## Функция, подсчета **ошибки**

In [ ]:
def dice_coef(y_true, y_pred):
  return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)

## Проверка работы сети 

In [ ]:
tags = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']

def recognizeSet(XX, YY, model, tags, length, value):
  correct_list = np.array([0] * 6) #Инициализируем массив правильных ответов в нули (сколько раз правильно определили класс)
  incorrect_list =  np.array([0] * 6)  #Инициализируем массив неправильных ответов в нули  (сколько раз неправильно определили класс)
  XX_array = XX
  YY_array = YY
  pred = model.predict(XX_array)
  pred[pred < value] = 0
  pred[pred > value] = 1

  for element in range(YY_array.shape[0]): # Проходим по всем примерам в батче
    for word in range(YY_array.shape[1]):  # Проходим по каждому слову
      for category in range(YY_array.shape[2]): # Проходим по каждой категории в слове
        if pred[element][word][category] == YY_array[element][word][category]: # Если предсказанное значение совпадает с истинным:
          correct_list[category] += 1 # Отмечаем, что мы правильно предсказали класс объекта
        else:  # Если предсказанное значение НЕ совпадает с истенным:
          incorrect_list[category] += 1 # Отмечаем, что мы не правильно предсказали класс объекта
      
  # Итоговая точность для каждого класса = кол. 100% * правильных/(кол. неправильных + кол. правильных)
  total = round(100*np.mean(correct_list/(correct_list + incorrect_list)),2) 
  return total

# 1. Сделайте свою собственную архитектуру и запустите, оцените точность на проверочной выборке.

Загрузка 

In [ ]:
# GENSIMtrainX = np.load('/content/drive/MyDrive/УИИ/Cегментация текста/GENSIMtrainX.npy')
# GENSIMtrainY = np.load('/content/drive/MyDrive/УИИ/Cегментация текста/GENSIMtrainY.npy')

# GENSIMtestX = np.load('/content/drive/MyDrive/УИИ/Cегментация текста/GENSIMtestX.npy')
# GENSIMtestY = np.load('/content/drive/MyDrive/УИИ/Cегментация текста/GENSIMtestY.npy')


In [ ]:
xLen = 256
embeddingSize = 300

In [ ]:
# Создание таблицы Pandas
df_result = pd.DataFrame(columns = ['val_loss', 'val_dice_coef', 'Средняя точность'])

## **Вариант 1**

Данная нейронная сеть будет лишь с **одним** пробросом между первыми и последними слоями. 

In [ ]:
def create_net_1(num_classes = 6, input_shape= (256, 300)):
  
  input = Input(input_shape)                  # (256, 300)

  # Block 1
  x = Conv1D(32, 3, padding='same')(input)    # (256, 32)
  x = BatchNormalization()(x)                 # (256, 32)
  x = Activation('relu')(x)                   # (256, 32)

  x = Conv1D(32, 3, padding='same')(x)        # (256, 32)
  x = BatchNormalization()(x)                 # (256, 32)
  block_1_out = Activation('relu')(x)         # (256, 32)

  x = MaxPooling1D()(block_1_out)             # (128, 32) !!! ->128


  # Block 2

  x = Conv1D(64, 3, padding='same')(x)        # (128, 64)
  x = BatchNormalization()(x)                 # (128, 64)
  x = Activation('relu')(x)                   # (128, 64)

  x = Conv1D(64, 3, padding='same')(x)        # (128, 64)
  x = BatchNormalization()(x)                 # (128, 64)
  x = Activation('relu')(x)                   # (128, 64)

  x = MaxPooling1D()(x)                       # (64, 64)


  # Block 3

  x = Conv1D(128, 3, padding='same')(x)       # (64, 128)
  x = BatchNormalization()(x)                 # (64, 128)
  x = Activation('relu')(x)                   # (64, 128)

  x = Conv1D(128, 3, padding='same')(x)       # (64, 128)
  x = BatchNormalization()(x)                 # (64, 128)
  x = Activation('relu')(x)                   # (64, 128)


  # UP 2

  x = Conv1DTranspose(64, 2, strides=2, padding='same')(x)  # (128, 64)
  x = BatchNormalization()(x)                               # (128, 64)
  x = Activation('relu')(x)                                 # (128, 64)

  x = Conv1D(64, 3, padding='same')(x)                      # (128, 64)
  x = BatchNormalization()(x)                               # (128, 64)
  x = Activation('relu')(x)                                 # (128, 64)

  x = Conv1D(64, 3, padding='same')(x)                      # (128, 64)   
  x = BatchNormalization()(x)                               # (128, 64)
  x = Activation('relu')(x)                                 # (128, 64)


  # UP 3

  x = Conv1DTranspose(32, 2, strides=2, padding='same')(x) # (256, 32)
  x = BatchNormalization()(x)                              # (256, 32)
  x = Activation('relu')(x)                                # (256, 32)

  x = concatenate([x, block_1_out])         # (256, 32) + (256, 32) = (256, 64)

  x = Conv1D(32, 3, padding='same')(x)      # (256, 32)
  x = BatchNormalization()(x)               # (256, 32)
  x = Activation('relu')(x)                 # (256, 32)

  x = Conv1D(32, 3, padding='same')(x)      # (256, 32)
  x = BatchNormalization()(x)               # (256, 32)
  x = Activation('relu')(x)                 # (256, 32)

  # Выходной

  x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)  # softmax
  # (256, 6)
  model = Model(input, x)

  model.compile(optimizer=Adadelta(learning_rate=0.0001), loss='categorical_crossentropy', metrics=[dice_coef]) #Adam

  return model

### **Создание модели**

In [ ]:
net_1 = create_net_1(input_shape=(xLen, embeddingSize))

### Просмотр модели

In [ ]:
net_1.summary(0)

In [ ]:
plot_model(net_1, dpi=70, show_shapes=True) 

### **Обучение**

In [ ]:
history_net_1 = net_1.fit(GENSIMtrainX, GENSIMtrainY, validation_data = (GENSIMtestX, GENSIMtestY), epochs=50, batch_size=64)

Epoch 1/50
214/214 [==============================] - 29s 47ms/step - loss: 1.5211 - dice_coef: 0.2017 - val_loss: 1.1779 - val_dice_coef: 0.1931
Epoch 2/50
214/214 [==============================] - 6s 27ms/step - loss: 1.5194 - dice_coef: 0.2019 - val_loss: 1.2650 - val_dice_coef: 0.1948
Epoch 3/50
214/214 [==============================] - 6s 27ms/step - loss: 1.5175 - dice_coef: 0.2020 - val_loss: 1.2936 - val_dice_coef: 0.1949
Epoch 4/50
214/214 [==============================] - 6s 27ms/step - loss: 1.5155 - dice_coef: 0.2022 - val_loss: 1.2961 - val_dice_coef: 0.1950
Epoch 5/50
214/214 [==============================] - 6s 28ms/step - loss: 1.5134 - dice_coef: 0.2024 - val_loss: 1.2949 - val_dice_coef: 0.1951
Epoch 6/50
214/214 [==============================] - 6s 27ms/step - loss: 1.5112 - dice_coef: 0.2025 - val_loss: 1.2931 - val_dice_coef: 0.1953
Epoch 7/50
214/214 [==============================] - 6s 27ms/step - loss: 1.5089 - dice_coef: 0.2027 - val_loss: 1.2917 - val_di

### **Проверка результатов и запись в таблицу**

In [ ]:
total_net_1 = recognizeSet(GENSIMtestX, GENSIMtestY, net_1, tags, xLen, 0.8) 

Сеть распознала категорию  'S1' с точностью в 69.57%
Сеть распознала категорию  'S2' с точностью в 97.98%
Сеть распознала категорию  'S3' с точностью в 87.24%
Сеть распознала категорию  'S4' с точностью в 86.23%
Сеть распознала категорию  'S5' с точностью в 93.46%
Сеть распознала категорию  'S6' с точностью в 99.91%
Средняя точность 89.07%


In [ ]:
df_result.loc['net_1'] = [min(history_net_1.history['val_loss']), min(history_net_1.history['val_dice_coef']), total_net_1]

In [ ]:
df_result.to_csv('/content/drive/MyDrive/УИИ/Cегментация текста/df_result.csv')
df_result

,val_loss,val_dice_coef,Средняя точность
net_1,1.177939,0.193128,89.07


## **Вариант 2** - Dense

В данном варианте после последнего **MaxPooling2D** будет слой **Dense**, перед этим вытянутый в вектор и далее снова преобразование в трехмерное пространство.

In [ ]:
def create_net_2(num_classes = 6, input_shape= (256, 300)):

  input = Input(input_shape)

  # Block 1
  x = Conv1D(32, 3, padding='same')(input)       # (256, 32)
  x = BatchNormalization()(x)                    # (256, 32)
  x = Activation('relu')(x)                      # (256, 32)

  x = Conv1D(32, 3, padding='same')(x)           # (256, 32) 
  x = BatchNormalization()(x)                    # (256, 32)
  block_1_out = Activation('relu')(x)            # (256, 32) ----> Выход

  x = MaxPooling1D()(block_1_out)                # (128, 32)

  # Block 2

  x = Conv1D(64, 3, padding='same')(x)           # (128, 64)
  x = BatchNormalization()(x)                    # (128, 64)
  x = Activation('relu')(x)                      # (128, 64)

  x = Conv1D(64, 3, padding='same')(x)           # (128, 64)
  x = BatchNormalization()(x)                    # (128, 64)
  x = Activation('relu')(x)                      # (128, 64)

  x = MaxPooling1D()(x)                          # (64, 64)


  # Block 3

  x = Conv1D(128, 3, padding='same')(x)          # (64, 128)
  x = BatchNormalization()(x)                    # (64, 128)
  x = Activation('relu')(x)                      # (64, 128)

  x = Conv1D(128, 3, padding='same')(x)          # (64, 128)
  x = BatchNormalization()(x)                    # (64, 128)
  x = Activation('relu')(x)                      # (64, 128)

  x = MaxPooling1D()(x)                          # (32, 128)

  # Block 4

  x = Conv1D(32, 3, padding='same')(x)          # (32, 32)
  x = BatchNormalization()(x)                   # (32, 32)
  x = Activation('relu')(x)                     # (32, 32)

  x = Conv1D(16, 3, padding='same')(x)          # (32, 16)
  x = BatchNormalization()(x)                   # (32, 16)
  x = Activation('relu')(x)                     # (32, 16)

  x = MaxPooling1D()(x)                         # (16, 16)

  # Block Dense

  x = Flatten()(x)                              # 256 (16*16)
  x = Dense(16*16)(x)                           # 256 (16*16)
  x = LeakyReLU()(x)                            # 256 (16*16)
  x = Reshape((16,16))(x)                       # (16, 16)
 
 
  # UP 2

  x = Conv1DTranspose(128, 2, strides=2, padding='same')(x)   # (32, 128)
  x = BatchNormalization()(x)                                 # (32, 128)
  x = Activation('relu')(x)                                   # (32, 128)

  x = Conv1D(128, 3, padding='same')(x)                       # (32, 128)
  x = BatchNormalization()(x)                                 # (32, 128)
  x = Activation('relu')(x)                                   # (32, 128)

  x = Conv1D(128, 3, padding='same')(x)                       # (32, 128)                         
  x = BatchNormalization()(x)                                 # (32, 128)
  x = Activation('relu')(x)                                   # (32, 128)

  # UP 3

  x = Conv1DTranspose(64, 2, strides=2, padding='same')(x)    # (64, 64)
  x = BatchNormalization()(x)                                 # (64, 64)
  x = Activation('relu')(x)                                   # (64, 64)

  x = Conv1D(64, 3, padding='same')(x)                        # (64, 64)
  x = BatchNormalization()(x)                                 # (64, 64)
  x = Activation('relu')(x)                                   # (64, 64)

  x = Conv1D(64, 3, padding='same')(x)                        # (64, 64)
  x = BatchNormalization()(x)                                 # (64, 64)
  x = Activation('relu')(x)                                   # (64, 64)

  # UP 4

  x = Conv1DTranspose(32, 2, strides=2, padding='same')(x)    # (128, 32)
  x = BatchNormalization()(x)                                 # (128, 32)
  x = Activation('relu')(x)                                   # (128, 32)

  x = Conv1D(32, 3, padding='same')(x)                        # (128, 32)
  x = BatchNormalization()(x)                                 # (128, 32)
  x = Activation('relu')(x)                                   # (128, 32)

  x = Conv1D(32, 3, padding='same')(x)                        # (128, 32)
  x = BatchNormalization()(x)                                 # (128, 32)
  x = Activation('relu')(x)                                   # (128, 32)

  # UP 5

  x = Conv1DTranspose(32, 2, strides=2, padding='same')(x)    # (256, 32)  
  x = BatchNormalization()(x)                                 # (256, 32)
  x = Activation('relu')(x)                                   # (256, 32)

  x = concatenate([x, block_1_out]) # Объединение слоев (256, 32) (256, 32) -> (256,64) 

  x = Conv1D(32, 3, padding='same')(x)                        # (256, 32)
  x = BatchNormalization()(x)                                 # (256, 32)
  x = Activation('relu')(x)                                   # (256, 32)

  x = Conv1D(32, 3, padding='same')(x)                        # (256, 32)
  x = BatchNormalization()(x)                                 # (256, 32)
  x = Activation('relu')(x)                                   # (256, 32)

  # Выходной

  x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)  # (176,240,6)
  # (256, 6)
  model = Model(input, x)

  model.compile(optimizer=Adadelta(learning_rate=0.0001), loss='categorical_crossentropy', metrics=[dice_coef])

  return model

### **Создание модели**

In [ ]:
net_2 = create_net_2(input_shape=(xLen, embeddingSize))

### Просмотр модели

In [ ]:
net_2.summary()

In [ ]:
plot_model(net_2, dpi=70, show_shapes=True) 

### **Обучение**

In [ ]:
history_net_2 = net_2.fit(GENSIMtrainX, GENSIMtrainY, validation_data = (GENSIMtestX, GENSIMtestY), epochs=50, batch_size=64)

Epoch 1/50
214/214 [==============================] - 30s 57ms/step - loss: 1.2869 - dice_coef: 0.2271 - val_loss: 1.1390 - val_dice_coef: 0.2005
Epoch 2/50
214/214 [==============================] - 8s 35ms/step - loss: 1.2860 - dice_coef: 0.2272 - val_loss: 1.1728 - val_dice_coef: 0.2041
Epoch 3/50
214/214 [==============================] - 8s 35ms/step - loss: 1.2849 - dice_coef: 0.2273 - val_loss: 1.2006 - val_dice_coef: 0.2032
Epoch 4/50
214/214 [==============================] - 8s 39ms/step - loss: 1.2841 - dice_coef: 0.2274 - val_loss: 1.2063 - val_dice_coef: 0.2031
Epoch 5/50
214/214 [==============================] - 8s 35ms/step - loss: 1.2827 - dice_coef: 0.2275 - val_loss: 1.2063 - val_dice_coef: 0.2032
Epoch 6/50
214/214 [==============================] - 7s 34ms/step - loss: 1.2817 - dice_coef: 0.2276 - val_loss: 1.2052 - val_dice_coef: 0.2033
Epoch 7/50
214/214 [==============================] - 7s 34ms/step - loss: 1.2804 - dice_coef: 0.2277 - val_loss: 1.2039 - val_di

### **Проверка результатов и запись в таблицу**

In [ ]:
total_net_2 = recognizeSet(GENSIMtestX, GENSIMtestY, net_2, tags, xLen, 0.8) 

Сеть распознала категорию  'S1' с точностью в 61.77%
Сеть распознала категорию  'S2' с точностью в 96.39%
Сеть распознала категорию  'S3' с точностью в 86.64%
Сеть распознала категорию  'S4' с точностью в 86.12%
Сеть распознала категорию  'S5' с точностью в 91.99%
Сеть распознала категорию  'S6' с точностью в 99.63%
Средняя точность 87.09%


In [ ]:
# Загрузка таблицы 
#df_result = pd.read_csv('/content/drive/MyDrive/УИИ/Cегментация текста/df_result.csv', index_col = 0)

In [ ]:
df_result.loc['net_2'] = [min(history_net_2.history['val_loss']), min(history_net_2.history['val_dice_coef']), total_net_2]
df_result.to_csv('/content/drive/MyDrive/УИИ/Cегментация текста/df_result.csv')


In [ ]:
df_result

,val_loss,val_dice_coef,Средняя точность
net_1,1.177939,0.193128,89.07
net_2,1.139000,0.200499,87.09


# 2. Возьмите лучшую архитектуру и проведите несколько экспериментов, меняя гиперпараметры выбранной архитектуры.

Как видно из таблицы Pandas первая нейронаня сеть показала результат лучше. Поэтому следующие эксперименты будут происходить на основе данной архитектуре. 

Всё обучение будет происходить в цикле, а результат записываться в таблицу Pandas. После обучения можно посомтреть какой подбор гиперпараметров дал более лучший результат. 

In [ ]:
GENSIMtrainX = np.load('/content/drive/MyDrive/УИИ/Cегментация текста/GENSIMtrainX.npy')
GENSIMtrainY = np.load('/content/drive/MyDrive/УИИ/Cегментация текста/GENSIMtrainY.npy')

GENSIMtestX = np.load('/content/drive/MyDrive/УИИ/Cегментация текста/GENSIMtestX.npy')
GENSIMtestY = np.load('/content/drive/MyDrive/УИИ/Cегментация текста/GENSIMtestY.npy')


In [ ]:
xLen = 256
embeddingSize = 300

In [ ]:
df = pd.DataFrame(columns = ['optimizer', 'avtivation_out', 'kernel_size', 'filters', 'Средняя точность'])
num = 1   # Счетчик экспериментов для записи в таблицу

In [ ]:
curr_time = time()

for optim in [Adadelta, Adam]:          # Оптимайзер
  for activ in ['sigmoid', 'softmax']:  # Активационная функция в выходном слое
    for kernel_size in [2, 3]:          # Ядро
      for filters in [4, 8, 16, 32]:    # Число фильтров

        fit_time = time()

        input = Input((256,300))  

        x = Conv1D(filters, kernel_size, padding='same')(input)    
        x = BatchNormalization()(x)                 
        x = Activation('relu')(x)                   

        x = Conv1D(filters, kernel_size, padding='same')(x)        
        x = BatchNormalization()(x)                
        block_1_out = Activation('relu')(x)         

        x = MaxPooling1D()(block_1_out)            

        x = Conv1D(filters*2, kernel_size, padding='same')(x) 
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv1D(filters*2, kernel_size, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = MaxPooling1D()(x)

        x = Conv1D(filters*4, kernel_size, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv1D(filters*4, kernel_size, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        # UP 

        x = Conv1DTranspose(filters*2, kernel_size, strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv1D(filters*2, kernel_size, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv1D(filters*2, kernel_size, padding='same')(x)   
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv1DTranspose(filters, 2, strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = concatenate([x, block_1_out])

        x = Conv1D(filters, kernel_size, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv1D(filters, kernel_size, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        # Выходной

        x = Conv1D(6, kernel_size, activation = activ, padding='same')(x)  # softmax
        
        model = Model(input, x)

        model.compile(optimizer=optim(learning_rate=0.0001), loss='categorical_crossentropy', metrics=[dice_coef])

        # Обучение
        history = model.fit(GENSIMtrainX, GENSIMtrainY, validation_data = (GENSIMtestX, GENSIMtestY), epochs=5, batch_size=64, verbose=0)

        # Проверка результата
        res = recognizeSet(GENSIMtestX, GENSIMtestY, model, tags, xLen, 0.8) 

        # Запись в таблицу
        if optim == Adadelta:
          df.loc[num] = ['Adadelta', activ, kernel_size, filters, res]
        else:
          df.loc[num] = ['Adam', activ, kernel_size, filters, res]

        df.to_csv('/content/drive/MyDrive/УИИ/Cегментация текста/df.csv')

        print(f"Время обучения {num} сети : {round(time() - fit_time, 2)} секунд")
        num += 1

print(f"Общее время составило: {round((time() - curr_time) / 60, 2)} минут")

Время обучения 1 сети : 24.06 секунды
Время обучения 2 сети : 16.77 секунды
Время обучения 3 сети : 15.07 секунды
Время обучения 4 сети : 15.66 секунды
Время обучения 5 сети : 16.84 секунды
Время обучения 6 сети : 12.84 секунды
Время обучения 7 сети : 14.03 секунды
Время обучения 8 сети : 15.34 секунды
Время обучения 9 сети : 11.66 секунды
Время обучения 10 сети : 12.49 секунды
Время обучения 11 сети : 11.67 секунды
Время обучения 12 сети : 16.68 секунды
Время обучения 13 сети : 11.65 секунды
Время обучения 14 сети : 12.41 секунды
Время обучения 15 сети : 12.73 секунды
Время обучения 16 сети : 12.07 секунды
Время обучения 17 сети : 16.11 секунды
Время обучения 18 сети : 12.54 секунды
Время обучения 19 сети : 12.26 секунды
Время обучения 20 сети : 12.27 секунды
Время обучения 21 сети : 12.0 секунды
Время обучения 22 сети : 16.1 секунды
Время обучения 23 сети : 16.82 секунды
Время обучения 24 сети : 12.02 секунды
Время обучения 25 сети : 12.26 секунды
Время обучения 26 сети : 12.04 секун

In [ ]:
df.to_csv('/content/drive/MyDrive/УИИ/Cегментация текста/df.csv')

In [ ]:
df

,optimizer,avtivation_out,kernel_size,filters,Средняя точность
1,Adadelta,sigmoid,2,4,88.97
2,Adadelta,sigmoid,2,8,89.15
3,Adadelta,sigmoid,2,16,88.96
4,Adadelta,sigmoid,2,32,88.60
5,Adadelta,sigmoid,3,4,88.53
6,Adadelta,sigmoid,3,8,89.09
7,Adadelta,sigmoid,3,16,88.19
8,Adadelta,sigmoid,3,32,88.10
9,Adadelta,softmax,2,4,89.17
10,Adadelta,softmax,2,8,89.17


# Выводы:

1. Для большего количества экспериментов обучение нейронных сетей было реализовано с помощью цикла.
2. Все данные были записаны в таблицу Pandas.
3. Основная проблема при обучение в цикле была нехватка памяти. Пришлось ограничить обучающую выборку в количестве 100 договоров.
4. Для достижения лучшего результата требуется большее число обучающих данных и соответствующие мощности. 
5. Так же требуется подбирать оптимальную архитектуру нейронных сетей. А так же гиперпараметры. 
6. В данной работе было изучено как разные архитектуры, а так же гиперпараметры влияют на конечный результат.
7. Использование таблиц помогают оценить результат.  